<a href="https://colab.research.google.com/github/Ritika101/Fashion-Recommendation-System/blob/master/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive 
drive.mount('/content/gdrive') 


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
# #reference: 
# #https://forums.fast.ai/t/how-to-find-similar-images-based-on-final-embedding-layer/16903

# # install fastai for the hook and annoy for approximate nearest neighbors 
!pip install fastai
!pip install annoy
!pip install gdown

     |████████████████████████████████| 645kB 3.4MB/s 
  Created wheel for annoy: filename=annoy-1.16.3-cp36-cp36m-linux_x86_64.whl size=297346 sha256=97557707d24c1e8dd97cc3de26e66f6f27f4543dc9f0caf17a54e1bbe2b66c68
  Stored in directory: /root/.cache/pip/wheels/f3/01/54/6ef760fe9f9fc6ba8c19cebbe6358212b5f3b5b0195c0b813f
Successfully built annoy


In [5]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import gdown
from fastai.vision import *
from fastai.metrics import accuracy, top_k_accuracy
from annoy import AnnoyIndex
import zipfile
import time
from google.colab import drive
%matplotlib inline

In [2]:
!pip install torchvision


In [4]:
import torch

In [ ]:
train_on_gpu = torch.cuda.is_available()
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ... To use GPU, go under edit > notebook settings')
else:
    print('CUDA is available!  Training on GPU ...')
    print(gpu_info)

CUDA is available!  Training on GPU ...
Wed Jun 24 12:04:13 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    27W / 149W |     11MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                

In [7]:
import gdown

In [11]:
# get the meta data
url = 'https://drive.google.com/file/d/0B7EVK8r0v71pWnFiNlNGTVloLUk/view?usp=sharing'
output = 'list_category_cloth.txt'
gdown.download(url, output, quiet=False)

url = 'https://drive.google.com/file/d/0B7EVK8r0v71pTGNoWkhZeVpzbFk/view?usp=sharing'
output = 'list_category_img.txt'
gdown.download(url, output, quiet=False)

url = 'https://drive.google.com/file/d/0B7EVK8r0v71pdS1FMlNreEwtc1E/view?usp=sharing'
output = 'list_eval_partition.txt'
gdown.download(url, output, quiet=False)

/usr/local/lib/python3.6/dist-packages/gdown/parse_url.py:31: UserWarning: You specified Google Drive Link but it is not the correct link to download the file. Maybe you should try: https://drive.google.com/uc?id=0B7EVK8r0v71pWnFiNlNGTVloLUk
  .format(url='https://drive.google.com/uc?id={}'.format(file_id))
Downloading...
From: https://drive.google.com/file/d/0B7EVK8r0v71pWnFiNlNGTVloLUk/view?usp=sharing
To: /content/list_category_cloth.txt
70.7kB [00:00, 14.9MB/s]
/usr/local/lib/python3.6/dist-packages/gdown/parse_url.py:31: UserWarning: You specified Google Drive Link but it is not the correct link to download the file. Maybe you should try: https://drive.google.com/uc?id=0B7EVK8r0v71pTGNoWkhZeVpzbFk
  .format(url='https://drive.google.com/uc?id={}'.format(file_id))
Downloading...
From: https://drive.google.com/file/d/0B7EVK8r0v71pTGNoWkhZeVpzbFk/view?usp=sharing
To: /content/list_category_img.txt
70.7kB [00:00, 13.2MB/s]
/usr/local/lib/python3.6/dist-packages/gdown/parse_url.py:31: 

'list_eval_partition.txt'

In [17]:
import gzipfile

ModuleNotFoundError: ignored

In [16]:
# get the images
root_path = './'
url = 'https://drive.google.com/file/d/0B7EVK8r0v71pa2EyNEJ0dE9zbU0/view?usp=sharing'
output = 'img.zip'
gdown.download(url, output, quiet=False)
with gzipfile.GzipFile("img.zip","r") as zip_ref:
    zip_ref.extractall(root_path)

/usr/local/lib/python3.6/dist-packages/gdown/parse_url.py:31: UserWarning: You specified Google Drive Link but it is not the correct link to download the file. Maybe you should try: https://drive.google.com/uc?id=0B7EVK8r0v71pa2EyNEJ0dE9zbU0
  .format(url='https://drive.google.com/uc?id={}'.format(file_id))
Downloading...
From: https://drive.google.com/file/d/0B7EVK8r0v71pa2EyNEJ0dE9zbU0/view?usp=sharing
To: /content/img.zip
70.1kB [00:00, 16.7MB/s]


NameError: ignored